### how to solve inter hyphen issue https://stackoverflow.com/questions/52293874/why-does-spacy-not-preserve-intra-word-hyphens-during-tokenization-like-stanford

In [3]:
import re
import os
import json
import spacy
import nltk
from spacy.tokenizer import Tokenizer
from spacy.util import compile_prefix_regex, compile_infix_regex, compile_suffix_regex
from benepar.spacy_plugin import BeneparComponent
from spacy.tokens import Doc
nlp = spacy.load('en')
nlp.add_pipe(BeneparComponent('benepar_en2'))

/home/pengfei/miniconda3/envs/benepar/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pengfei/miniconda3/envs/benepar/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pengfei/miniconda3/envs/benepar/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pengfei/miniconda3/envs/benepar

In [4]:
# def custom_tokenizer(nlp):
# #     infix_re = re.compile(r'''[~-]''')
#     infix_re = compile_infix_regex(nlp.Defaults.infixes)
#     prefix_re = compile_prefix_regex(nlp.Defaults.prefixes)
#     suffix_re = compile_suffix_regex(nlp.Defaults.suffixes)

#     return Tokenizer(nlp.vocab, 
#                      prefix_search=prefix_re.search,
#                                 suffix_search=suffix_re.search,
#                                 infix_finditer=infix_re.finditer,
#                                 token_match=None)


def custom_tokenizer(text):
    tokens = nltk.word_tokenize(text)
    # with this:
    return Doc(nlp.vocab, tokens)

nlp.tokenizer = custom_tokenizer

In [5]:
def get_doc(doc_path):
    with open(doc_path) as f:
        txt = f.read().replace('\n', ' ')
    doc = nlp(txt)
    sentences = []
    sent_token_offset = 0
    for i, sent in enumerate(doc.sents):
        if i == 0: continue
        words = []
        dependencies = []
        count = 0
        for word in sent:
            charoffsetbegin = word.idx
            charoffsetend = word.idx + len(word)
            word_pos = word.pos_
            word_string = str(word)
            if str(word) == '\"':
                if count % 2 == 0:
                    word_string = '``'
                else:
                    word_string = '\'\''
            word_combo = [word_string, 
                          {"CharacterOffsetBegin": charoffsetbegin, 
                                      "CharacterOffsetEnd": charoffsetend, 
                                      "Linkers": [], 
                                      "PartOfSpeech": word.pos_}]
            words.append(word_combo)

            dep_r = word.dep_
            parent = str(word.head) + '-' + str(word.head.i - sent_token_offset)
            child = str(word) + '-' + str(word.i - sent_token_offset)
            dep = [dep_r, parent, child]
            dependencies.append(dep)

        parsetree = sent._.parse_string
        parsetree = re.sub(r'\(_SP ((.|\n)*)\)', '', parsetree)
        sentences.append({"parsetree": parsetree, "words": words, "dependencies": dependencies})
        sent_token_offset += len(sent)
    return sentences

In [6]:
folder_path = '/home/pengfei/data/pdtb3-dataset/all/raw/'
files = os.listdir(folder_path)
all_data = {}
files = [f for f in files if f[4:6] in ['01', '00', '24']]
files = ['wsj_0203',
 'wsj_0205',
 'wsj_0206',
 'wsj_0212',
 'wsj_0217',
 'wsj_0226',
 'wsj_0234',
 'wsj_0249',
 'wsj_0285',
 'wsj_0320',
 'wsj_0330',
 'wsj_0366',
 'wsj_0381',
 'wsj_0382',
 'wsj_0383',
 'wsj_0384',
 'wsj_0385',
 'wsj_0386',
 'wsj_0387',
 'wsj_0388',
 'wsj_0389',
 'wsj_0390',
 'wsj_0391',
 'wsj_0392',
 'wsj_0393',
 'wsj_0394',
 'wsj_0395',
 'wsj_0396',
 'wsj_0397',
 'wsj_0398',
 'wsj_0399',
 'wsj_0425',
 'wsj_0432',
 'wsj_0438',
 'wsj_0454',
 'wsj_0455',
 'wsj_0459',
 'wsj_0463',
 'wsj_0468',
 'wsj_0506',
 'wsj_0513',
 'wsj_0517',
 'wsj_0538',
 'wsj_0543',
 'wsj_0549',
 'wsj_0555',
 'wsj_0591',
 'wsj_0620',
 'wsj_0650',
 'wsj_0678',
 'wsj_0685',
 'wsj_0691',
 'wsj_0698',
 'wsj_0699',
 'wsj_0703',
 'wsj_0707',
 'wsj_0711',
 'wsj_0715',
 'wsj_0716',
 'wsj_0720',
 'wsj_0721',
 'wsj_0724',
 'wsj_0749',
 'wsj_0821',
 'wsj_0822',
 'wsj_0823',
 'wsj_0824',
 'wsj_0825',
 'wsj_0826',
 'wsj_0827',
 'wsj_0828',
 'wsj_0829',
 'wsj_0830',
 'wsj_0831',
 'wsj_0832',
 'wsj_0833',
 'wsj_0834',
 'wsj_0835',
 'wsj_0836',
 'wsj_0837',
 'wsj_0838',
 'wsj_0839',
 'wsj_0840',
 'wsj_0841',
 'wsj_0842',
 'wsj_0843',
 'wsj_0844',
 'wsj_0845',
 'wsj_0846',
 'wsj_0847',
 'wsj_0848',
 'wsj_0849',
 'wsj_0850',
 'wsj_0851',
 'wsj_0852',
 'wsj_0853',
 'wsj_0854',
 'wsj_0855',
 'wsj_0856',
 'wsj_0857',
 'wsj_0858',
 'wsj_0859',
 'wsj_0860',
 'wsj_0861',
 'wsj_0862',
 'wsj_0863',
 'wsj_0864',
 'wsj_0865',
 'wsj_0866',
 'wsj_0867',
 'wsj_0868',
 'wsj_0869',
 'wsj_0870',
 'wsj_0871',
 'wsj_0872',
 'wsj_0873',
 'wsj_0874',
 'wsj_0875',
 'wsj_0876',
 'wsj_0877',
 'wsj_0878',
 'wsj_0879',
 'wsj_0880',
 'wsj_0881',
 'wsj_0882',
 'wsj_0883',
 'wsj_0884',
 'wsj_0885',
 'wsj_0886',
 'wsj_0887',
 'wsj_0888',
 'wsj_0889',
 'wsj_0890',
 'wsj_0891',
 'wsj_0892',
 'wsj_0893',
 'wsj_0894',
 'wsj_0895',
 'wsj_0896',
 'wsj_0897',
 'wsj_0898',
 'wsj_0899',
 'wsj_0901',
 'wsj_0920',
 'wsj_0921',
 'wsj_0947',
 'wsj_0951',
 'wsj_0998',
 'wsj_1009',
 'wsj_1063',
 'wsj_1067',
 'wsj_1077',
 'wsj_1085',
 'wsj_1087',
 'wsj_1088',
 'wsj_1090',
 'wsj_1098',
 'wsj_1104',
 'wsj_1114',
 'wsj_1115',
 'wsj_1129',
 'wsj_1197',
 'wsj_1202',
 'wsj_1240',
 'wsj_1245',
 'wsj_1277',
 'wsj_1326',
 'wsj_1342',
 'wsj_1344',
 'wsj_1378',
 'wsj_1384',
 'wsj_1385',
 'wsj_1393',
 'wsj_1400',
 'wsj_1406',
 'wsj_1420',
 'wsj_1425',
 'wsj_1427',
 'wsj_1444',
 'wsj_1482',
 'wsj_1559',
 'wsj_1576',
 'wsj_1579',
 'wsj_1608',
 'wsj_1625',
 'wsj_1638',
 'wsj_1653',
 'wsj_1716',
 'wsj_1771',
 'wsj_1785',
 'wsj_1801',
 'wsj_1805',
 'wsj_1807',
 'wsj_1834',
 'wsj_1836',
 'wsj_1838',
 'wsj_1845',
 'wsj_1933',
 'wsj_1942',
 'wsj_1954',
 'wsj_1955',
 'wsj_1956',
 'wsj_1958',
 'wsj_1979',
 'wsj_1993',
 'wsj_1995',
 'wsj_2035',
 'wsj_2050',
 'wsj_2090',
 'wsj_2097',
 'wsj_2117',
 'wsj_2122',
 'wsj_2134',
 'wsj_2137',
 'wsj_2147',
 'wsj_2170',
 'wsj_2171',
 'wsj_2173',
 'wsj_2174',
 'wsj_2175',
 'wsj_2176',
 'wsj_2177',
 'wsj_2178',
 'wsj_2179',
 'wsj_2180',
 'wsj_2181',
 'wsj_2182',
 'wsj_2183',
 'wsj_2184',
 'wsj_2185',
 'wsj_2186',
 'wsj_2187',
 'wsj_2188',
 'wsj_2189',
 'wsj_2190',
 'wsj_2191',
 'wsj_2192',
 'wsj_2193',
 'wsj_2194',
 'wsj_2195',
 'wsj_2196',
 'wsj_2197',
 'wsj_2198',
 'wsj_2199',
 'wsj_2220',
 'wsj_2236',
 'wsj_2263',
 'wsj_2279',
 'wsj_2283',
 'wsj_2284',
 'wsj_2285',
 'wsj_2286',
 'wsj_2287',
 'wsj_2288',
 'wsj_2289',
 'wsj_2290',
 'wsj_2291',
 'wsj_2292',
 'wsj_2293',
 'wsj_2294',
 'wsj_2295',
 'wsj_2296',
 'wsj_2297',
 'wsj_2298',
 'wsj_2299',
 'wsj_2304',
 'wsj_2307',
 'wsj_2312',
 'wsj_2318',
 'wsj_2349',
 'wsj_2385',
 'wsj_2388',
 'wsj_2389',
 'wsj_2390']

In [7]:
all_data = {}
for f in files:
    doc_path = os.path.join(folder_path, f)
    doc = get_doc(doc_path)
    all_data[f] = {'sentences': doc}

In [8]:
with open('pdtb-parse.json', 'w') as f:
    json.dump(all_data, f)

In [171]:
# json_path = '/home/pengfei/data/pdtb3-dataset/all/conll/train/pdtb-parses.json'
# with open(json_path,  'r') as f:
#     parse_dict = json.load(f)